In [56]:
import pandas as pd
import numpy as np
import pickle
import time

In [57]:
label_list = [0, 1, 2, 3, 4]

In [78]:
# Choose if you want to look at EEG or EMG data

data_type = "EEG"
data_type = "EMG"

In [79]:
# choose individuum
subject = "m292"

In [80]:
# Load persistence diagrams

train_persistence_diagrams = {} # dictionary with labels as keys, persistence diagrams of the respective classes as values
test_persistence_diagrams = {} # dictionary with labels as keys, persistence diagrams of the respective classes as values


for label in label_list:
    train_persistence_diagrams[label] = np.load("Embeddings_and_Persistence_Diagrams/"+str(subject)+"/Train/"+str(data_type)+"/PD"+str(label)+".npy", allow_pickle=True)
    test_persistence_diagrams[label] = np.load("Embeddings_and_Persistence_Diagrams/"+str(subject)+"/Test/"+str(data_type)+"/PD"+str(label)+".npy", allow_pickle=True)

In [81]:
# Adcock-Carlson coordinates

In [82]:
def carlsson_coordinates(persistence_diagrams):
    n = len(persistence_diagrams)
    persistence_diagrams_features_cc1 = np.zeros(n)
    persistence_diagrams_features_cc2 = np.zeros(n)
    persistence_diagrams_features_cc3 = np.zeros(n)
    persistence_diagrams_features_cc4 = np.zeros(n)
    start = time.time()
    ymax = 0
    for i in range(0,n):
        if len(persistence_diagrams[i])>0:
            b = np.max(persistence_diagrams[i][:,1])
        else:
            b = ymax
        if ymax < b:
            ymax = b
        else:
            ymax = ymax
    for i in range(0,n):
        if len(persistence_diagrams[i])>0:
            x = persistence_diagrams[i][:,0]
            y = persistence_diagrams[i][:,1]
            persistence_diagrams_features_cc1[i] = sum(x*(y-x))
            persistence_diagrams_features_cc2[i] = sum((ymax - y)*(y-x))
            persistence_diagrams_features_cc3[i] = sum(x**2*(y-x)**4)
            persistence_diagrams_features_cc4[i] = sum((ymax-y)**2*(y-x)**4)
        else:
            persistence_diagrams_features_cc1[i] = 0
            persistence_diagrams_features_cc2[i] = 0
            persistence_diagrams_features_cc3[i] = 0
            persistence_diagrams_features_cc4[i] = 0
            
    return persistence_diagrams_features_cc1, persistence_diagrams_features_cc2, persistence_diagrams_features_cc3, persistence_diagrams_features_cc4

In [83]:
def create_dataframe(data_type, X_features_cc1, X_features_cc2, X_features_cc3, X_features_cc4, label, train):
    feature_df = pd.DataFrame()

    feature_df[str(data_type)+"_AC_coordinate_cc1"] = X_features_cc1
    feature_df[str(data_type)+"_AC_coordinate_cc2"] = X_features_cc2
    feature_df[str(data_type)+"_AC_coordinate_cc3"] = X_features_cc3
    feature_df[str(data_type)+"_AC_coordinate_cc4"] = X_features_cc4
    feature_df["Label"] = label

    # Subject
    feature_df["Subject"] = subject

    # Training or test set (boolean)
    feature_df["Train"] = train

    return feature_df

In [84]:
train_features_dataframes = {}
test_features_dataframes = {}


for label in label_list:
    
    # Train data Adcock Carlson coordinates 
    train_coord_1, train_coord_2, train_coord_3, train_coord_4 = carlsson_coordinates(train_persistence_diagrams[label])
    train_features_dataframes[label] = create_dataframe(data_type, train_coord_1, train_coord_2, train_coord_3, \
                                                        train_coord_4, label, True)

    # Test data Adcock Carlson coordinates 
    test_coord_1, test_coord_2, test_coord_3, test_coord_4 = carlsson_coordinates(test_persistence_diagrams[label])
    test_features_dataframes[label] = create_dataframe(data_type, test_coord_1, test_coord_2, test_coord_3, \
                                                       test_coord_4, label, False)

In [85]:
# Concatenate and save dataframes
train_feature_df = pd.concat([train_features_dataframes[0], train_features_dataframes[1], train_features_dataframes[2], \
                              train_features_dataframes[3], train_features_dataframes[4]], ignore_index=True)

test_feature_df = pd.concat([test_features_dataframes[0], test_features_dataframes[1], test_features_dataframes[2], \
                             test_features_dataframes[3], test_features_dataframes[4]], ignore_index=True)

train_feature_df.to_csv("Features/"+str(subject)+"/Train/"+str(data_type)+"/Advanced_Features.csv")
test_feature_df.to_csv("Features/"+str(subject)+"/Test/"+str(data_type)+"/Advanced_Features.csv")